In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import Image 
import scipy
# Define directories
train_dir = "C:/Users/ADMIN/Desktop/train"
test_dir = "C:/Users/ADMIN/Desktop/test"

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Model Building
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=20
)

# Model Evaluation
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')

# Save the model
model.save('cancer_classification_model.h5')


Found 11879 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Epoch 1/20
371/371 [==============================] - 435s 1s/step - loss: 0.4864 - accuracy: 0.7656 - val_loss: 0.4459 - val_accuracy: 0.7777
Epoch 2/20
371/371 [==============================] - 326s 877ms/step - loss: 0.4008 - accuracy: 0.8227 - val_loss: 0.3605 - val_accuracy: 0.8584
Epoch 3/20
371/371 [==============================] - 316s 850ms/step - loss: 0.3747 - accuracy: 0.8330 - val_loss: 0.3943 - val_accuracy: 0.8180
Epoch 4/20
371/371 [==============================] - 318s 858ms/step - loss: 0.3683 - accuracy: 0.8399 - val_loss: 0.2872 - val_accuracy: 0.8931
Epoch 5/20
371/371 [==============================] - 326s 879ms/step - loss: 0.3532 - accuracy: 0.8448 - val_loss: 0.3869 - val_accuracy: 0.8145
Epoch 6/20
371/371 [==============================] - 346s 931ms/step - loss: 0.3422 - accuracy: 0.8519 - val_loss: 0.3139 - val_accuracy: 0.8921
Epoch 7/20
371/371 [==============================] - 336s 906ms/step - loss: 0.3326 - accuracy: 0.8550 - val_loss: 0.2728 - va

c:\Users\ADMIN\anaconda3\envs\compgraphics\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
model.save('cancer_prediction_model.keras')

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('cancer_prediction_model.keras')

# Function to preprocess the image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(150, 150))  # Resize the image to 150x150 pixels
    img_array = img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add an extra dimension for batch size
    img_array /= 255.0  # Scale the pixel values to [0, 1]
    return img_array

# Function to predict the class of the image
def predict_image(image_path):
    processed_image = preprocess_image(image_path)
    prediction = model.predict(processed_image)
    class_label = 'malignant' if prediction[0][0] > 0.5 else 'benign'
    return class_label

# Test the function
image_path = "C:/Users/ADMIN/Pictures/Screenshots/Screenshot 2024-07-03 232757.png"
result = predict_image(image_path)
print(f'The image is predicted to be: {result}')


1/1 [==============================] - 0s 100ms/step
The image is predicted to be: benign
